In [1]:
import gym
import vh_graph
import time
import json
from virtualhome.simulation.unity_simulator.comm_unity import UnityCommunication

In [2]:
print('Imports done')
env = gym.make('vh_graph-v0')
print('Env created')
state_path = 'example_graph/example_graph.json'
task_goals = {0: '(inside cup[2009] bathroom[1])', 1: '(ontop video_game_controller[2006] kitchen_counter_[230])'}
comm = UnityCommunication()

Imports done
Env created


In [3]:
print('Restart...')
comm.reset(0)
s, g = comm.environment_graph()
s = env.reset(g)
env.to_pomdp()

Restart...


In [8]:
script = ['[walk] <milk> (1)',
          '[grab] <milk> (1)',
          '[walk] <microwave> (1)',
          '[open] <microwave> (1)',
          '[putin] <milk> (1) <microwave> (1)',
          '[close] <microwave> (1)',
          '[switchon] <microwave> (1)']
          
for sc in script:
    rewards, states, infos = env.step({0: sc})
    print("Rewards:", rewards)
    objects, predicates = env.get_objects_and_predicates(0)
    print("Objects:", objects)
    print([str(p) for p in predicates])
    #{'image_width': 300, 'image_height': 300, 'mode': 'normal'}
    env.get_image(comm.camera_count()[1], 0, 6, 1, infos, comm)
    print('***********************************************')

Rewards: [0]
Objects: [('character', 1), ('bathroom', 11), ('rug', 30), ('curtains', 31), ('curtains', 32), ('curtains', 33), ('ceilinglamp', 34), ('walllamp', 35), ('walllamp', 36), ('walllamp', 37), ('bathtub', 38), ('towelrack', 39), ('towelrack', 40), ('towelrack', 41), ('towelrack', 42), ('wallshelf', 43), ('stall', 44), ('toilet', 45), ('stall', 46), ('curtains', 47), ('bathroomcabinet', 48), ('bathroomcounter', 49), ('faucet', 50), ('door', 51), ('doorjamb', 52), ('towel', 53), ('towel', 54), ('towel', 55), ('perfume', 56), ('deodorant', 57), ('hairproduct', 58), ('hairproduct', 59), ('facecream', 60), ('plate', 61), ('toothpaste', 62), ('painkillers', 63), ('waterglass', 64), ('toothbrush', 65), ('barsoap', 66), ('towel', 67), ('towel', 68), ('candle', 69), ('window', 70), ('lightswitch', 71), ('washingmachine', 72), ('bedroom', 73), ('window', 91), ('ceilinglamp', 101), ('tablelamp', 102), ('tablelamp', 103), ('nightstand', 104), ('bookshelf', 105), ('nightstand', 106), ('chai

UnboundLocalError: local variable 'image_width' referenced before assignment

In [ ]:
comm.reset(0)

# Get nodes for salmon and microwave, glass, faucet and sink
salmon_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'salmon'][0]
microwave_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'microwave'][0]
glass_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'waterglass'][-1]
sink_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'sink'][0]
faucet_id = [node['id'] for node in g['nodes'] if node['class_name'] == 'faucet'][-1]


# Put salmon in microwave
script = [
    '<char0> [walk] <salmon> ({}) | <char1> [walk] <glass> ({})'.format(salmon_id, glass_id),
    '<char0> [grab] <salmon> ({}) | <char1> [grab] <glass> ({})'.format(salmon_id, glass_id),
    '<char0> [open] <microwave> ({}) | <char1> [walk] <sink> ({})'.format(microwave_id, sink_id),
    '<char0> [putin] <salmon> ({}) <microwave> ({}) | <char1> [putback] <glass> ({}) <sink> ({})'.format(salmon_id, microwave_id, glass_id, sink_id),
    '<char0> [close] <microwave> ({}) | <char1> [switchon] <faucet> ({})'.format(microwave_id, faucet_id)
]

s, cc = comm.camera_count()
for script_instruction in script:
    print(script_instruction)
    comm.render_script([script_instruction], recording=False, skip_animation=True)
    # Here you can get an observation, for instance
    s, im = comm.camera_image([cc-3], image_width=300, image_height=300)